In [1]:
import os
import copy

In [2]:
def estimate_emission_parameters(training_set, k_value=1):
    state_count = {}
    state_observation_count = {}
    estimated_emission_parameters = {}
    #Get a set of the trained words
    trained_words = set()
    
    #k_value: k occurences of generating observation #UNK# from any label y
    
    for i in range(len(training_set)):
        #if its not a single empty line that separates sentences
        if(len(training_set[i])!=0):
            parts = training_set[i].split(" ")
            observation = ' '.join(parts[:len(parts)-1])
            state = parts[-1]
            
            #Increment count(y->x)
            if (observation,state) in state_observation_count:
                state_observation_count[(observation,state)]+=1
            else:
                state_observation_count[(observation,state)]=1

            #Increment count(y)
            if state in state_count:
                state_count[state]+=1
            else:
                state_count[state]=1
                
            trained_words.add(observation)
        else:
            continue
            
    #For each x|y, calculate count(y->x)/(count(y)+k) and calculate k/(count(y)+k) -> the x for this is #UNK#
    #We assume from any label y there is a certain chance of generating #UNK# as a rare event,
    #and emprically we assume we have observed that there are k occurences of such an event 
    for k,v in state_observation_count.items():
        estimated_emission_parameters[k] = v/(state_count[k[1]]+k_value)
        estimated_emission_parameters[("#UNK#",k[1])] = k_value/(state_count[k[1]]+k_value)
    
    return estimated_emission_parameters, list(trained_words)

In [3]:
#Function to create a list of lists where each list contains only the states of a sentence
def sentence_creator_states(original):
    sentences = []
    sentence = []
    for i in original:
        #If it is not an empty line
        if i!='':
            #Append the state to sentence
            parts = i.split(" ")
            state = parts[-1]
            sentence.append(state)
        #If it is an empty line
        #The sentence is complete
        else:
            #Add the START and STOP state for computation later
            sentence.insert(0, "START")
            sentence.append("STOP")
            sentences.append(sentence)
            sentence = []
            
    #In the case of the last sentence where it the training set does not end with an empty line
    if len(sentence)!=0:
        sentences.append(sentence)
    
    return sentences

In [4]:
def estimate_transition_parameters(training_set):
    #Break the training set into sentences, where each sentence contains its states
    training_set = sentence_creator_states(training_set)
    state_count = {}
    state_transition_count = {}
    estimated_transition_parameters = {}
    
    #For each sentence (list of states)
    for sentence in training_set:
        #For each state in the sentence
        for i in range(len(sentence)):
            #Compute the counts up until the STOP state
            if(i!=len(sentence)-1):
                #Increment count(y_i-1 -> y_i)
                if (sentence[i], sentence[i+1]) in state_transition_count:
                    state_transition_count[(sentence[i], sentence[i+1])]+=1
                else:
                    state_transition_count[(sentence[i], sentence[i+1])]=1

                #Increment count(y_i-1)
                if sentence[i] in state_count:
                    state_count[sentence[i]]+=1
                else:
                    state_count[sentence[i]]=1
                    
    #For each y_i|y_i-1, calculate count(y_i-1 -> y_i)/count(y_i-1)
    for k,v in state_transition_count.items():
        estimated_transition_parameters[k] = v/state_count[k[0]]
    
    return estimated_transition_parameters, list(state_count.keys())

### Learn emission and transition parameters from training set

#### Read training set

In [5]:
filepath_ES_train = os.path.join(os.getcwd(), 'Data', 'ES', 'train')

#Read the file contents
with open(filepath_ES_train, 'r', encoding='utf-8') as file:
    file_contents_ES_train = file.readlines()
    
#Convert to training set
es_training_set = [w.strip() for w in file_contents_ES_train]

In [6]:
#Calculate the parameters using the training set
estimated_emission_parameters,trained_words = estimate_emission_parameters(es_training_set)

In [7]:
estimated_emission_parameters 

{('Estuvimos', 'O'): 0.00020664003306240529,
 ('#UNK#', 'O'): 3.444000551040088e-05,
 ('hace', 'O'): 0.0008954401432704229,
 ('poco', 'O'): 0.0018942003030720485,
 ('mi', 'O'): 0.0024796803967488635,
 ('pareja', 'O'): 0.00044772007163521146,
 ('y', 'O'): 0.0352665656426505,
 ('yo', 'O'): 0.0012398401983744318,
 ('comiendo', 'O'): 0.00034440005510400884,
 ('resultó', 'O'): 0.00013776002204160352,
 ('todo', 'O'): 0.0039606006336961016,
 ('muy', 'O'): 0.013638242182118749,
 ('bien', 'O'): 0.005682600909216145,
 (',', 'O'): 0.05730816916930707,
 ('tanto', 'O'): 0.0013431602149056344,
 ('la', 'O'): 0.026002204160352666,
 ('comida', 'B-positive'): 0.14556416881998277,
 ('#UNK#', 'B-positive'): 0.0008613264427217916,
 ('el', 'O'): 0.022110483537677365,
 ('vino', 'B-positive'): 0.00516795865633075,
 ('trato', 'B-positive'): 0.03789836347975883,
 ('decoración', 'B-positive'): 0.006029285099052541,
 ('…', 'O'): 0.0015498002479680396,
 ('nos', 'O'): 0.005028240804518529,
 ('gustó', 'O'): 0.000378

In [8]:
estimated_transition_parameters, all_states = estimate_transition_parameters(es_training_set)

In [9]:
estimated_transition_parameters

{('START', 'O'): 0.9289176090468497,
 ('O', 'O'): 0.8856896848630963,
 ('O', 'B-positive'): 0.03650766316514551,
 ('B-positive', 'O'): 0.871551724137931,
 ('O', 'STOP'): 0.06344067504735663,
 ('O', 'B-negative'): 0.012226623041157224,
 ('B-negative', 'O'): 0.8110236220472441,
 ('START', 'B-positive'): 0.052234787291330104,
 ('O', 'B-neutral'): 0.0021353538832443605,
 ('B-neutral', 'I-neutral'): 0.20833333333333334,
 ('I-neutral', 'I-neutral'): 0.6511627906976745,
 ('I-neutral', 'O'): 0.3488372093023256,
 ('B-positive', 'I-positive'): 0.11637931034482758,
 ('I-positive', 'I-positive'): 0.5700636942675159,
 ('I-positive', 'O'): 0.4267515923566879,
 ('B-neutral', 'O'): 0.7916666666666666,
 ('B-negative', 'I-negative'): 0.1784776902887139,
 ('I-negative', 'O'): 0.39766081871345027,
 ('START', 'B-negative'): 0.014001077005923533,
 ('I-negative', 'I-negative'): 0.6023391812865497,
 ('START', 'B-neutral'): 0.004846526655896607,
 ('B-positive', 'STOP'): 0.008620689655172414,
 ('B-negative', 'S

In [10]:
all_states

['START',
 'O',
 'B-positive',
 'B-negative',
 'B-neutral',
 'I-neutral',
 'I-positive',
 'I-negative']

### Implement Viterbi Algorithm

#### Read ES dev.in Dataset for evaluation

In [11]:
filepath_ES_devin = os.path.join(os.getcwd(), 'Data', 'ES', 'dev.in')

#Read the file contents
with open(filepath_ES_devin, 'r', encoding='utf-8') as file:
    file_contents_ES_devin = file.readlines()
    
es_devin = [w.strip() for w in file_contents_ES_devin]

#### Convert ES dev.in to a list of lists where each list is a sentence of observations

In [12]:
#Function to create a list of lists where each list contains only the observations of a sentence
def sentence_creator_observations(original):
    sentences = []
    sentence = []
    for i in original:
        if i!='':
            sentence.append(i)
        else:
            sentences.append(sentence)
            sentence = []
            
    if len(sentence)!=0:
        sentences.append(sentence)
    
    return sentences

In [13]:
es_devin = sentence_creator_observations(es_devin)

In [14]:
es_devin

[['Plato',
  'degustación',
  ':',
  'un',
  'poco',
  'abundante',
  'de',
  'más',
  ',',
  'pero',
  'bien',
  'cocinado',
  '.'],
 ['restaurante', 'excelente', 'con', 'carne', 'de', 'alta', 'calidad', '.'],
 ['Las',
  'posibilidades',
  'en',
  'el',
  'restaurante',
  'son',
  'fundamentalmente',
  'tres',
  ';',
  'carta',
  'normal',
  ',',
  'menú',
  'degustacion',
  'y',
  'una',
  'opción',
  'intermedia',
  'que',
  'es',
  'una',
  'selección',
  'de',
  'primeros',
  'y',
  'postres',
  'y',
  'carta',
  'para',
  'el',
  'segundo',
  '.'],
 ['No', 'perderse', 'el', 'sorbete', 'de', 'mojito', '.'],
 ['para', 'mi', 'perfecto', '!'],
 ['Devolucion',
  'a',
  'cocina',
  ',',
  'amabilidad',
  'de',
  'camarera',
  ',',
  'requerimiento',
  'de',
  'cuenta',
  'y',
  'adios',
  '.'],
 ['Así',
  'como',
  'el',
  'romesco',
  ',',
  'que',
  'era',
  'un',
  'poco',
  '"',
  'de',
  'bote',
  '"',
  '.'],
 ['Destacar',
  'los',
  'arroces',
  ',',
  'la',
  'caldereta',
  'de

### Test Vertibi Algorithm with Toy Example

#### Get the first sentence

In [15]:
sentence = es_devin[0]

In [16]:
sentence

['Plato',
 'degustación',
 ':',
 'un',
 'poco',
 'abundante',
 'de',
 'más',
 ',',
 'pero',
 'bien',
 'cocinado',
 '.']

In [17]:
all_states

['START',
 'O',
 'B-positive',
 'B-negative',
 'B-neutral',
 'I-neutral',
 'I-positive',
 'I-negative']

#### Append STOP state

In [18]:
all_states.append('STOP')

In [19]:
all_states

['START',
 'O',
 'B-positive',
 'B-negative',
 'B-neutral',
 'I-neutral',
 'I-positive',
 'I-negative',
 'STOP']

#### Deep copy learnt parameters from ES train earlier

In [20]:
emission_parameters = copy.deepcopy(estimated_emission_parameters)

In [21]:
transition_parameters = copy.deepcopy(estimated_transition_parameters)

#### Pad the sentence to account for START and STOP state

In [22]:
#Pad each sentence to account for position 0 (start) and position n+1 (stop)
sentence.insert(0, "padding")
sentence.append("padding")

In [23]:
sentence

['padding',
 'Plato',
 'degustación',
 ':',
 'un',
 'poco',
 'abundante',
 'de',
 'más',
 ',',
 'pero',
 'bien',
 'cocinado',
 '.',
 'padding']

#### Initialise Score Matrix

In [24]:
#Initialise a score matrix to store the highest scoring paths 
#from START to state u at position j
#index by [position,state]
#number of states T
#number of positions 0 to n+1
#dimensions: number of positions x number of states
score_matrix = [[0 for _ in range(len(all_states))] for _ in range(len(sentence))] 

In [25]:
score_matrix

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

#### Forward Pass

In [26]:
#0. Map each state to an index number
all_states_map = {k:v for v,k in enumerate(all_states)}

In [27]:
all_states_map

{'START': 0,
 'O': 1,
 'B-positive': 2,
 'B-negative': 3,
 'B-neutral': 4,
 'I-neutral': 5,
 'I-positive': 6,
 'I-negative': 7,
 'STOP': 8}

#### Initialisation Step of Forward Pass

In [28]:
#1. Initialisation Step
for state in all_states:
    if(state=='START'):
        score_matrix[0][all_states_map[state]] = 1
    else:
        score_matrix[0][all_states_map[state]] = 0

In [29]:
score_matrix

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

#### Forward Pass from j=0 to j=n-1

In [30]:
#2. Forward Pass from j=0 to j=n-1 (inclusive)
#0 1 n-1 n n+1
#len = 5
#len-2 = 3
#For each position j from 0 to n-1
for position in range(len(sentence)-2):
    #For each state u belonging to T except for START and STOP
    for state in all_states:
        if(state=='START' or state=='STOP'):
            continue
        score = 0
        #For each prev state v belonging to T
        for prev_state in all_states:
            temp = 0
            #If the word in j+1 position has been trained before
            if(sentence[position+1] in trained_words):
                #If the emission and transition has been trained before
                if((sentence[position+1],state) in emission_parameters.keys() and (prev_state,state) in transition_parameters.keys()):
                    #Calculate: score of prev_state v in position j * emission prob of observation from curr_state u * transiton prob of prev_state v to curr_state u
                    temp = score_matrix[position][all_states_map[prev_state]]*emission_parameters[(sentence[position+1],state)]*transition_parameters[(prev_state,state)]
            else:
                #If the emission and transition has been trained before
                if(("#UNK#",state) in emission_parameters.keys() and (prev_state,state) in transition_parameters.keys()):
                    #Calculate: score of prev_state v in position j * emission prob of #UNK# from curr_state u * transiton prob of prev_state v to curr_state u
                    temp = score_matrix[position][all_states_map[prev_state]]*emission_parameters[("#UNK#",state)]*transition_parameters[(prev_state,state)]
            #Store the score of the highest scoring path (max v) from START to this node (position j+1, state u) 
            if(temp>score):
                score = temp
                score_matrix[position+1][all_states_map[state]] = score

In [31]:
score_matrix

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3.1991927574281916e-05, 0, 3.6652034046920245e-05, 0, 0, 0, 0, 0],
 [0, 9.213768724107643e-09, 0, 0, 0, 0, 0, 7.606477187299569e-08, 0],
 [0, 7.083835938704282e-11, 0, 0, 0, 0, 0, 0, 0],
 [0, 7.886896794885997e-13, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.3231639086808406e-15, 0, 0, 0, 0, 0, 0, 0],
 [0, 5.246888045503111e-19, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.618071662883478e-20, 0, 0, 0, 0, 0, 0, 0],
 [0, 3.55365323892177e-23, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.8037368111227872e-24, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.0508756639119981e-26, 5.671853224723894e-29, 0, 0, 0, 0, 0, 0],
 [0, 5.289079293776172e-29, 3.30448016938397e-31, 0, 0, 0, 0, 0, 0],
 [0, 8.066680970038135e-33, 0, 0, 0, 0, 0, 0, 0],
 [0, 3.993541483983454e-34, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

#### Final Step of Forward Pass

In [32]:
#3. Final Step
#0 1 n-1 n n+1
#len = 5
#len-2 = 3
#len-1 = 4
score = 0
#For each prev state v belonging to T
for prev_state in all_states:
    temp = 0
    #If the transition has been trained before
    if((prev_state,'STOP') in transition_parameters.keys()):
        #Calculate: score of prev_state v in position n * transiton prob of prev_state v to STOP state
        temp = score_matrix[len(sentence)-2][all_states_map[prev_state]]*transition_parameters[(prev_state,'STOP')]

    #Store the score of the highest scoring path (max v) from START to STOP 
    if(temp>score):
        score = temp
        score_matrix[len(sentence)-1][all_states_map['STOP']] = score

In [33]:
score_matrix

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 3.1991927574281916e-05, 0, 3.6652034046920245e-05, 0, 0, 0, 0, 0],
 [0, 9.213768724107643e-09, 0, 0, 0, 0, 0, 7.606477187299569e-08, 0],
 [0, 7.083835938704282e-11, 0, 0, 0, 0, 0, 0, 0],
 [0, 7.886896794885997e-13, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.3231639086808406e-15, 0, 0, 0, 0, 0, 0, 0],
 [0, 5.246888045503111e-19, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.618071662883478e-20, 0, 0, 0, 0, 0, 0, 0],
 [0, 3.55365323892177e-23, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.8037368111227872e-24, 0, 0, 0, 0, 0, 0, 0],
 [0, 1.0508756639119981e-26, 5.671853224723894e-29, 0, 0, 0, 0, 0, 0],
 [0, 5.289079293776172e-29, 3.30448016938397e-31, 0, 0, 0, 0, 0, 0],
 [0, 8.066680970038135e-33, 0, 0, 0, 0, 0, 0, 0],
 [0, 3.993541483983454e-34, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 2.533529675735327e-35]]

#### Backward Pass

#### Initialise a list to store the optimal ys found in the backwards pass

In [34]:
optimal_y = [None for _ in range(len(sentence))]

In [35]:
optimal_y

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [36]:
#len(sentence)-1: last index
#Calculate y_n*
#at position n
score = 0
for state in all_states:
    temp = 0
    if(state=='START' or state=='STOP'):
        continue
    #If the transition has been trained before
    if((state,'STOP') in transition_parameters.keys()):
        temp = score_matrix[len(sentence)-2][all_states_map[state]] * transition_parameters[(state,'STOP')]
    if(temp>score):
        score = temp
        optimal_y[len(sentence)-2] = state

#Calculate y_j*
#from position n-1 to position 1 (inclusive)
for j in range(len(sentence)-3,0,-1):
    score = 0
    for state in all_states:
        temp = 0
        if(state=='START' or state=='STOP'):
            continue
        #If the transition has been trained before: from this state to the optimal state found in the next position
        if((state,optimal_y[j+1]) in transition_parameters.keys()):
            temp = score_matrix[j][all_states_map[state]] * transition_parameters[(state,optimal_y[j+1])] 
        if(temp>score):
            score = temp
            optimal_y[j] = state

In [37]:
optimal_y

[None,
 'B-negative',
 'I-negative',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 None]

In [38]:
sentence

['padding',
 'Plato',
 'degustación',
 ':',
 'un',
 'poco',
 'abundante',
 'de',
 'más',
 ',',
 'pero',
 'bien',
 'cocinado',
 '.',
 'padding']

#### Remove the added padding from the sentence

In [39]:
#Remove the padding
sentence.pop(0)
sentence.pop(len(sentence)-1)

'padding'

#### Remove position 0 and position n+1

In [40]:
#Remove position 0 and position n+1
optimal_y.pop(0)
optimal_y.pop(len(optimal_y)-1)

In [41]:
sentence

['Plato',
 'degustación',
 ':',
 'un',
 'poco',
 'abundante',
 'de',
 'más',
 ',',
 'pero',
 'bien',
 'cocinado',
 '.']

In [42]:
optimal_y

['B-negative',
 'I-negative',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

#### Join the sentence with its predicted states

In [43]:
tagged_sentence = [f"{sentence[i]} {optimal_y[i]}" for i in range(len(sentence))]

In [44]:
tagged_sentence

['Plato B-negative',
 'degustación I-negative',
 ': O',
 'un O',
 'poco O',
 'abundante O',
 'de O',
 'más O',
 ', O',
 'pero O',
 'bien O',
 'cocinado O',
 '. O']